# Working with multiple files

On occasion, we will need to combine more than 2 files using some combination of `UNION` and `JOIN`.  In this lecture, we will show a clean approach to scaling up these operations up to any number of files.  In the process, we will

1. Use `list` comprehensions to process and `UNION` many similar files.
2. Use `dict` comprehensions to store and access many tables by name.

In [1]:
import polars as pl
from glob import glob

## The Basics of working with many files.

* Use `glob.glob` to find all files that match a pattern
* Convert all files to `pd.DataFrames`
* Store the `df` in a list or dictionary

### What the heck is a `glob`

`glob.glob`

* Takes a path regular expression
* Returns a list of files that match the patterm
* Relative paths!

In [2]:
sales_files = glob('./data/auto_sales_*.csv')
sales_files

['./data/auto_sales_apr.csv', './data/auto_sales_may.csv']

### Search tools for `glob`

* Use `*` as a wildcard,
* Use `?` for optional characters, and
* Use `[...]` to define character classes.

In [3]:
(baseball_core_csv := 
 glob('./data/baseballdatabank*/core/*csv', recursive=True)
)

['./data/baseballdatabank-2023.1/core/Managers.csv',
 './data/baseballdatabank-2023.1/core/Fielding.csv',
 './data/baseballdatabank-2023.1/core/Parks.csv',
 './data/baseballdatabank-2023.1/core/People.csv',
 './data/baseballdatabank-2023.1/core/PitchingPost.csv',
 './data/baseballdatabank-2023.1/core/Teams.csv',
 './data/baseballdatabank-2023.1/core/Appearances.csv',
 './data/baseballdatabank-2023.1/core/TeamsFranchises.csv',
 './data/baseballdatabank-2023.1/core/Batting.csv',
 './data/baseballdatabank-2023.1/core/ManagersHalf.csv',
 './data/baseballdatabank-2023.1/core/FieldingOF.csv',
 './data/baseballdatabank-2023.1/core/Pitching.csv',
 './data/baseballdatabank-2023.1/core/HomeGames.csv',
 './data/baseballdatabank-2023.1/core/BattingPost.csv',
 './data/baseballdatabank-2023.1/core/TeamsHalf.csv',
 './data/baseballdatabank-2023.1/core/SeriesPost.csv',
 './data/baseballdatabank-2023.1/core/FieldingPost.csv',
 './data/baseballdatabank-2023.1/core/AllstarFull.csv',
 './data/baseballdata

### Recursive search with `glob`

To search all folders and sub-folders, you need to
1. Set the `recursive=True` option in `glob`, and
2. Use `**` to represent one or more folders.

In [4]:
(all_baseball_csv 
 := glob('./data/baseballdatabank*/**/*.csv', recursive=True)
)

['./data/baseballdatabank-2023.1/core/Managers.csv',
 './data/baseballdatabank-2023.1/core/Fielding.csv',
 './data/baseballdatabank-2023.1/core/Parks.csv',
 './data/baseballdatabank-2023.1/core/People.csv',
 './data/baseballdatabank-2023.1/core/PitchingPost.csv',
 './data/baseballdatabank-2023.1/core/Teams.csv',
 './data/baseballdatabank-2023.1/core/Appearances.csv',
 './data/baseballdatabank-2023.1/core/TeamsFranchises.csv',
 './data/baseballdatabank-2023.1/core/Batting.csv',
 './data/baseballdatabank-2023.1/core/ManagersHalf.csv',
 './data/baseballdatabank-2023.1/core/FieldingOF.csv',
 './data/baseballdatabank-2023.1/core/Pitching.csv',
 './data/baseballdatabank-2023.1/core/HomeGames.csv',
 './data/baseballdatabank-2023.1/core/BattingPost.csv',
 './data/baseballdatabank-2023.1/core/TeamsHalf.csv',
 './data/baseballdatabank-2023.1/core/SeriesPost.csv',
 './data/baseballdatabank-2023.1/core/FieldingPost.csv',
 './data/baseballdatabank-2023.1/core/AllstarFull.csv',
 './data/baseballdata

## Using `polars` built-in `glob`

* `pl.read_csv( ..., glob=True)` is default,
* Will search for all files when given a wildcard/optional/class, and
* UNION the resulting tables.

**Note.** All information in the file name is lost!

In [5]:
sales_files = pl.read_csv('./data/auto_sales_*.csv')
sales_files

shape: (8, 6)
┌─────┬─────────────┬─────────┬───────┬─────┬───────┐
│     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck │
│ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   │
│ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   │
╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╡
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
│ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    │
│ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    │
│ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     │
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    │
│ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    │
│ 2   ┆ Yolanda     ┆ 19      ┆ 10    ┆ 28  ┆ 17    │
│ 3   ┆ Xerxes      ┆ 11      ┆ 27    ┆ 17  ┆ 9     │
└─────┴─────────────┴─────────┴───────┴─────┴───────┘

#### BEWARE - Files need to be UNIONABLE

In [34]:
pl.read_csv('./data/baseball/core/*.csv')

ComputeError: schema lengths differ

## Saving information in the path

Often, there is data contained in the path/filename of the file.  To save this information, it is useful to

1. Use `with_columns` to make a `_path` column containing this information,
2. Use `polars` string function to extract any needed information from this column.

### Example - Using `glob` to read and UNION the sales data, including the month

Using `glob` with a `list` to automate reading an combining files 

#### Step 1 - Get the file names

In [7]:
from glob import glob

(sales_files := 
 glob('./data/auto_sales_*.csv')
)

['./data/auto_sales_apr.csv', './data/auto_sales_may.csv']

#### Step 2 - Read the files into a list of data frames with a `_path` columns

In [9]:
(sales_by_month := 
 [pl.read_csv(p).with_columns(_path = pl.lit(p)) for p in sales_files]
)

[shape: (4, 7)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────────────────────────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ _path                     │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---                       │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ str                       │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╪═══════════════════════════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ ./data/auto_sales_apr.csv │
 │ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    ┆ ./data/auto_sales_apr.csv │
 │ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    ┆ ./data/auto_sales_apr.csv │
 │ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     ┆ ./data/auto_sales_apr.csv │
 └─────┴─────────────┴─────────┴───────┴─────┴───────┴───────────────────────────┘,
 shape: (4, 7)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────────────────────────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ _

 #### Inspect each data from with head

In [10]:
[df.head(2) for df in sales_by_month]

[shape: (2, 7)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────────────────────────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ _path                     │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---                       │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ str                       │
 ╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╪═══════════════════════════╡
 │ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ ./data/auto_sales_apr.csv │
 │ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    ┆ ./data/auto_sales_apr.csv │
 └─────┴─────────────┴─────────┴───────┴─────┴───────┴───────────────────────────┘,
 shape: (2, 7)
 ┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────────────────────────┐
 │     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ _path                     │
 │ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---                       │
 │ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ s

#### Inspecting the `shape`s

In [11]:
[df.shape for df in sales_by_month]

[(4, 7), (4, 7)]

#### Step 3 - Union the data frames using `pl.concat`

In [12]:
(all_sales :=
 pl.concat(sales_by_month)
)

shape: (8, 7)
┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────────────────────────┐
│     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ _path                     │
│ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---                       │
│ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ str                       │
╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╪═══════════════════════════╡
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ ./data/auto_sales_apr.csv │
│ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    ┆ ./data/auto_sales_apr.csv │
│ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    ┆ ./data/auto_sales_apr.csv │
│ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     ┆ ./data/auto_sales_apr.csv │
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ ./data/auto_sales_may.csv │
│ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    ┆ ./data/auto_sales_may.csv │
│ 2   ┆ Yolanda     ┆ 19      ┆ 10    ┆ 28  ┆ 17    ┆ ./data/auto_sales_may.csv │
│ 3   ┆ Xerxes      ┆ 11      ┆ 27    ┆ 17  ┆ 9     ┆ ./data/auto_sales_may.csv │
└─────┴─────────────┴─────────┴───────┴─────┴───────┴───────────────────────────┘

#### Step 4 - Extract the month from the path using `polars` string helper methods 

**Approach 1.** Split and get the month.

In [22]:
(all_sales_w_month :=
 all_sales
 .with_columns(month = (pl.col('_path')
                          .str.split('_')
                          .list.get(-1)
                          .str.split('.')
                          .list.get(0)
                       )
              )
 .drop('_path')
)

shape: (8, 7)
┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────┐
│     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ month │
│ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---   │
│ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ str   │
╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╪═══════╡
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ apr   │
│ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    ┆ apr   │
│ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    ┆ apr   │
│ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     ┆ apr   │
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ may   │
│ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    ┆ may   │
│ 2   ┆ Yolanda     ┆ 19      ┆ 10    ┆ 28  ┆ 17    ┆ may   │
│ 3   ┆ Xerxes      ┆ 11      ┆ 27    ┆ 17  ┆ 9     ┆ may   │
└─────┴─────────────┴─────────┴───────┴─────┴───────┴───────┘

**Approach 2.** Use a RegEx to extract the month.

In [24]:
(all_sales_w_month :=
 all_sales
 .with_columns(month = pl.col('_path').str.extract(r'\./data/auto_sales_([a-z]+)\.csv'))
 .drop('_path')
)


shape: (8, 7)
┌─────┬─────────────┬─────────┬───────┬─────┬───────┬───────┐
│     ┆ Salesperson ┆ Compact ┆ Sedan ┆ SUV ┆ Truck ┆ month │
│ --- ┆ ---         ┆ ---     ┆ ---   ┆ --- ┆ ---   ┆ ---   │
│ i64 ┆ str         ┆ i64     ┆ i64   ┆ i64 ┆ i64   ┆ str   │
╞═════╪═════════════╪═════════╪═══════╪═════╪═══════╪═══════╡
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ apr   │
│ 1   ┆ Bob         ┆ 19      ┆ 12    ┆ 17  ┆ 20    ┆ apr   │
│ 2   ┆ Yolanda     ┆ 19      ┆ 8     ┆ 32  ┆ 15    ┆ apr   │
│ 3   ┆ Xerxes      ┆ 12      ┆ 23    ┆ 18  ┆ 9     ┆ apr   │
│ 0   ┆ Ann         ┆ 22      ┆ 18    ┆ 15  ┆ 12    ┆ may   │
│ 1   ┆ Bob         ┆ 20      ┆ 14    ┆ 6   ┆ 24    ┆ may   │
│ 2   ┆ Yolanda     ┆ 19      ┆ 10    ┆ 28  ┆ 17    ┆ may   │
│ 3   ┆ Xerxes      ┆ 11      ┆ 27    ┆ 17  ┆ 9     ┆ may   │
└─────┴─────────────┴─────────┴───────┴─────┴───────┴───────┘

### Dealing with Windows paths

* Windows uses backslash `\` instead of forward slash `/` to separate folders/files.
* Even on Windows, `glob` understands unix style paths that use `/` to separate files/folders.
* Since `\` is the escape character in Python, Windows paths will contain an escaped/literal backslash `\\`.
* The wild card parts of the pattern will return Windows style paths with `\\`

In [2]:
# Non-recursive search run on Windows
baseball_core_csv = glob('./data/baseballdatabank*/core/*csv', recursive=True)

baseball_core_csv

['./data\\baseballdatabank-2023.1\\core\\AllstarFull.csv',
 './data\\baseballdatabank-2023.1\\core\\Appearances.csv',
 './data\\baseballdatabank-2023.1\\core\\Batting.csv',
 './data\\baseballdatabank-2023.1\\core\\BattingPost.csv',
 './data\\baseballdatabank-2023.1\\core\\Fielding.csv',
 './data\\baseballdatabank-2023.1\\core\\FieldingOF.csv',
 './data\\baseballdatabank-2023.1\\core\\FieldingOFsplit.csv',
 './data\\baseballdatabank-2023.1\\core\\FieldingPost.csv',
 './data\\baseballdatabank-2023.1\\core\\HomeGames.csv',
 './data\\baseballdatabank-2023.1\\core\\Managers.csv',
 './data\\baseballdatabank-2023.1\\core\\ManagersHalf.csv',
 './data\\baseballdatabank-2023.1\\core\\Parks.csv',
 './data\\baseballdatabank-2023.1\\core\\People.csv',
 './data\\baseballdatabank-2023.1\\core\\Pitching.csv',
 './data\\baseballdatabank-2023.1\\core\\PitchingPost.csv',
 './data\\baseballdatabank-2023.1\\core\\SeriesPost.csv',
 './data\\baseballdatabank-2023.1\\core\\Teams.csv',
 './data\\baseballdataba

In [3]:
# Recursive search run on Windows
all_baseball_csv = glob('./data/baseballdatabank*/**/*.csv', recursive=True)

all_baseball_csv

['./data\\baseballdatabank-2023.1\\contrib\\AwardsManagers.csv',
 './data\\baseballdatabank-2023.1\\contrib\\AwardsPlayers.csv',
 './data\\baseballdatabank-2023.1\\contrib\\AwardsShareManagers.csv',
 './data\\baseballdatabank-2023.1\\contrib\\AwardsSharePlayers.csv',
 './data\\baseballdatabank-2023.1\\contrib\\CollegePlaying.csv',
 './data\\baseballdatabank-2023.1\\contrib\\HallOfFame.csv',
 './data\\baseballdatabank-2023.1\\contrib\\Salaries.csv',
 './data\\baseballdatabank-2023.1\\contrib\\Schools.csv',
 './data\\baseballdatabank-2023.1\\core\\AllstarFull.csv',
 './data\\baseballdatabank-2023.1\\core\\Appearances.csv',
 './data\\baseballdatabank-2023.1\\core\\Batting.csv',
 './data\\baseballdatabank-2023.1\\core\\BattingPost.csv',
 './data\\baseballdatabank-2023.1\\core\\Fielding.csv',
 './data\\baseballdatabank-2023.1\\core\\FieldingOF.csv',
 './data\\baseballdatabank-2023.1\\core\\FieldingOFsplit.csv',
 './data\\baseballdatabank-2023.1\\core\\FieldingPost.csv',
 './data\\baseballda

### Extracting information from Windows paths

On Windows, we need to switch `/` to `\\`

#### Example 1 - Split and get the file name in Windows

In [4]:
get_file_name = lambda p: p.split('\\')[-1].split('.')[0]

[get_file_name(p) for p in all_baseball_csv]


['AwardsManagers',
 'AwardsPlayers',
 'AwardsShareManagers',
 'AwardsSharePlayers',
 'CollegePlaying',
 'HallOfFame',
 'Salaries',
 'Schools',
 'AllstarFull',
 'Appearances',
 'Batting',
 'BattingPost',
 'Fielding',
 'FieldingOF',
 'FieldingOFsplit',
 'FieldingPost',
 'HomeGames',
 'Managers',
 'ManagersHalf',
 'Parks',
 'People',
 'Pitching',
 'PitchingPost',
 'SeriesPost',
 'Teams',
 'TeamsFranchises',
 'TeamsHalf',
 'Teams']

#### Example 2 - Using a regular expression to extract file name in Windows

In [5]:
file_name = re.compile(r'./data\\baseballdatabank-2023.1\\[a-z]+\\([a-zA-Z]+).csv')

get_file_names = lambda paths: [_match.group(1) for p in paths if (_match := file_name.match(p))]

get_file_names(all_baseball_csv)

['AwardsManagers',
 'AwardsPlayers',
 'AwardsShareManagers',
 'AwardsSharePlayers',
 'CollegePlaying',
 'HallOfFame',
 'Salaries',
 'Schools',
 'AllstarFull',
 'Appearances',
 'Batting',
 'BattingPost',
 'Fielding',
 'FieldingOF',
 'FieldingOFsplit',
 'FieldingPost',
 'HomeGames',
 'Managers',
 'ManagersHalf',
 'Parks',
 'People',
 'Pitching',
 'PitchingPost',
 'SeriesPost',
 'Teams',
 'TeamsFranchises',
 'TeamsHalf',
 'Teams']

## Store in `dict` or `list`?

* Natural sequence/order? $\rightarrow$ `list`
    *  Example: Lakes data and years are a natural sequence
* Easier to refer by name? $\rightarrow$ `dict`
    * Baseball files have no order and easier to refer to by name

## <font color="red"> Exercise 3.1</font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Use `glob` to get all 6 file paths.
2. Use a list comprehension to create a list of data frames, including a `_path` column containing the corresponding path to that file.
3. Use `pd.concat` to combine these 6 data frames into one combined `df`
4. Extract the month from the `_path` column, then drop the `_path` column.


In [46]:
# Your code here